# EDS Block Testing - Browser Only

This notebook is designed for **browser execution only** using the ipynb-viewer block.

## How to Use

1. **View on EDS site**: Add the ipynb-viewer block pointing to this notebook
2. **Run any cell**: Click "Run" buttons to execute code cells
3. **Import helpers as needed**: Each cell imports what it needs

## Important Notes

- **Browser-only execution**: Runs in your browser via ipynb-viewer block
- **Direct imports**: Each cell imports what it needs - no initialization required
- **Interactive previews**: Opens overlay with live block decoration
- **Cell independence**: Run any cell at any time in any order

## Part 1: Simple DOM Test

Test basic DOM functionality using native browser APIs.

In [1]:
// Create a simple DOM element
const testDiv = document.createElement('div');
testDiv.textContent = 'Hello from Browser!';
testDiv.className = 'test';
testDiv.style.cssText = 'padding: 20px; background: #e3f2fd; border-radius: 8px; color: #1976d2; font-weight: bold;';

console.log('✓ Created element:', testDiv.outerHTML);

return testDiv.outerHTML;

2:17 - Cannot find name 'document'. Do you need to change your target library? Try changing the 'lib' compiler option to include 'dom'.
9:1 - A 'return' statement can only be used within a function body.


## Part 2: Testing Blocks

Test EDS blocks by importing helper functions.

In [ ]:
// Test HelloWorld block
const { testBlock } = await import('/scripts/ipynb-helpers.js');

const helloBlock = await testBlock('helloworld');
console.log('✓ HelloWorld block created');

return helloBlock.outerHTML;

## Part 3: Before and After Transformation

See how blocks transform DOM structure.

In [ ]:
// Test Accordion block - BEFORE and AFTER
const { testBlock } = await import('/scripts/ipynb-helpers.js');

const accordionContent = `
  <div>
    <div>What is EDS?</div>
    <div>Edge Delivery Services is Adobe's modern web platform.</div>
  </div>
  <div>
    <div>How do blocks work?</div>
    <div>Blocks transform DOM elements using JavaScript decoration.</div>
  </div>
`;

// BEFORE
const before = document.createElement('div');
before.className = 'accordion';
before.innerHTML = accordionContent;
const beforeHTML = before.innerHTML.trim().substring(0, 100);

// AFTER
const after = await testBlock('accordion', accordionContent);
const afterHTML = after.innerHTML.trim().substring(0, 100);

return `BEFORE:\n${beforeHTML}...\n\nAFTER:\n${afterHTML}...\n\nTransformed: ${before.children.length} divs → ${after.querySelectorAll('details').length} details`;

## Part 4: Visual Preview with Overlay

Create visual previews that open in an overlay with full styling!

**Note**: Press ESC or click outside to close the overlay.

In [ ]:
// Create visual preview in overlay
const { showPreview } = await import('/scripts/ipynb-helpers.js');

const accordionContent = `
  <div>
    <div>What is EDS?</div>
    <div>Edge Delivery Services is Adobe's modern web platform for building fast, performant websites.</div>
  </div>
  <div>
    <div>How do blocks work?</div>
    <div>Blocks are JavaScript functions that decorate DOM elements and transform content structure.</div>
  </div>
  <div>
    <div>Why use ipynb-viewer?</div>
    <div>The ipynb-viewer block lets end users interact with executable notebooks directly on your EDS site.</div>
  </div>
`;

// Opens overlay with styled block!
await showPreview('accordion', accordionContent);

return '✓ Preview overlay opened';

## Part 5: Available Blocks

List of common blocks you can test in this project.

In [2]:
// Common blocks available for testing
const commonBlocks = [
  'accordion',
  'cards',
  'columns',
  'fragment',
  'header',
  'footer',
  'hero',
  'helloworld',
  'ipynb-viewer'
];

console.log('Available blocks:', commonBlocks.join(', '));

return commonBlocks;

16:1 - A 'return' statement can only be used within a function body.


## Part 6: Pure JavaScript

Pure JavaScript examples that work in the browser.

In [ ]:
// Simple calculation
const a = 10;
const b = 20;
const sum = a + b;

console.log('Sum:', sum);
console.log('Product:', a * b);
console.log('Average:', sum / 2);

return sum;

In [ ]:
// Array operations
const numbers = [1, 2, 3, 4, 5];
const doubled = numbers.map(n => n * 2);
const sumNums = numbers.reduce((acc, n) => acc + n, 0);

console.log('Original:', numbers);
console.log('Doubled:', doubled);
console.log('Sum:', sumNums);

return doubled;

In [ ]:
// Text transformations
const text = 'hello world from jupyter';
const upper = text.toUpperCase();
const titleCase = text.replace(/\b\w/g, c => c.toUpperCase());

console.log('Original:', text);
console.log('Uppercase:', upper);
console.log('Title Case:', titleCase);

return titleCase;

## Quick Reference

### Import Helper Functions

Import what you need in each cell:

```javascript
const { testBlock, showPreview } = await import('/scripts/ipynb-helpers.js');
```

### Available Functions

**Test blocks:**
```javascript
const block = await testBlock('blockname', '<div>content</div>');
```

**Show preview (opens overlay):**
```javascript
await showPreview('blockname', '<div>content</div>');
```

### Code Pattern

Cell code runs in async context - just write naturally:

```javascript
const { testBlock, showPreview } = await import('/scripts/ipynb-helpers.js');
const block = await testBlock('blockname', '<div>content</div>');
await showPreview('blockname', '<div>content</div>');
return block.outerHTML;
```

### Preview Features

- Opens in overlay on same page
- Full CSS styling
- Block JavaScript decoration
- Interactive elements work
- Press ESC or click outside to close
- Multiple previews replace previous one

### Tips

✅ Import helpers in each cell that needs them
✅ Write code naturally with `await` - it runs in async context
✅ Use `return` to display results in output cell
✅ Check console.log for debugging
✅ Press ESC or click backdrop to close overlay
✅ Each cell is independent - run any cell at any time